# Panel Unit Root Tests: A Complete Guide

This tutorial demonstrates how to test for unit roots in panel data using advanced econometric tests.

## Overview

Testing for unit roots is crucial before estimating panel models:

- **Stationary data**: Use level regressions
- **Non-stationary data**: Consider differencing or cointegration analysis

## Available Tests

PanelBox implements several panel unit root tests:

### H0: Unit Root (Traditional Tests)

1. **Breitung (2000)**: Robust to heterogeneity in intercepts and trends
2. **IPS (2003)**: Allows heterogeneous AR coefficients across entities
3. **LLC (2002)**: Assumes common AR coefficient

### H0: Stationarity (Complementary Test)

4. **Hadri (2000)**: LM test with reversed null hypothesis

## Example: Testing Purchasing Power Parity (PPP)

We'll test whether real exchange rates are stationary, which is a key prediction of PPP theory.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from panelbox.diagnostics.unit_root import (
    hadri_test,
    breitung_test,
    panel_unit_root_test
)

# Set random seed for reproducibility
np.random.seed(42)

## 1. Generate Simulated Exchange Rate Data

We'll create a panel of real exchange rates for 20 countries over 200 quarters.

Under PPP, real exchange rates should be stationary (mean-reverting).
We'll simulate data that exhibits *slow* mean reversion.

In [ ]:
def generate_exchange_rate_data(n_countries=20, n_periods=200, mean_reversion=0.95):
    """
    Generate simulated real exchange rate data.
    
    Parameters:
    -----------
    mean_reversion : float
        AR(1) coefficient. Values close to 1 indicate slow mean reversion.
        - mean_reversion < 1: Stationary (PPP holds)
        - mean_reversion = 1: Unit root (PPP fails)
    """
    data = []
    
    for country in range(n_countries):
        # Generate AR(1) process: q_t = μ + ρ*q_{t-1} + ε_t
        q = np.zeros(n_periods)
        q[0] = np.random.randn() * 0.2  # Initial value
        
        for t in range(1, n_periods):
            # Mean reversion toward equilibrium (q = 0)
            q[t] = mean_reversion * q[t-1] + np.random.randn() * 0.05
        
        # Add to dataframe
        for t in range(n_periods):
            data.append({
                'country': country,
                'quarter': t,
                'real_exchange_rate': q[t]
            })
    
    return pd.DataFrame(data)

# Generate data with slow mean reversion (ρ = 0.95)
df = generate_exchange_rate_data(mean_reversion=0.95)
print(df.head(10))
print(f"\nPanel dimensions: {df['country'].nunique()} countries × {df['quarter'].nunique()} quarters")

## 2. Visual Inspection

Let's plot the real exchange rates for a few countries:

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for i, country in enumerate(range(6)):
    country_data = df[df['country'] == country]
    axes[i].plot(country_data['quarter'], country_data['real_exchange_rate'])
    axes[i].axhline(y=0, color='r', linestyle='--', alpha=0.3)
    axes[i].set_title(f'Country {country}')
    axes[i].set_xlabel('Quarter')
    axes[i].set_ylabel('Real Exchange Rate')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservation: Series appear to fluctuate around zero, suggesting possible stationarity.")
print("However, the persistence is high - formal testing is needed.")

## 3. Run Individual Tests

### 3.1 Hadri (2000) Test: H0 = Stationarity

In [ ]:
# Hadri test with constant only
hadri_result = hadri_test(
    df,
    variable='real_exchange_rate',
    entity_col='country',
    time_col='quarter',
    trend='c',
    robust=True
)

print(hadri_result.summary())

### 3.2 Breitung (2000) Test: H0 = Unit Root

In [ ]:
# Breitung test with constant and trend
breitung_result = breitung_test(
    df,
    variable='real_exchange_rate',
    entity_col='country',
    time_col='quarter',
    trend='ct'
)

print(breitung_result.summary())

## 4. Unified Test: Run All Tests Simultaneously

The `panel_unit_root_test()` function runs multiple tests and provides a comparative summary:

In [ ]:
# Run all available tests
result = panel_unit_root_test(
    df,
    variable='real_exchange_rate',
    entity_col='country',
    time_col='quarter',
    test='all',  # Run all available tests
    trend='c'
)

print(result.summary_table())

## 5. Sensitivity Analysis: Different Trend Specifications

Results can be sensitive to the choice of deterministic terms:

In [ ]:
# Test with constant only
result_c = panel_unit_root_test(df, 'real_exchange_rate', 
                                  entity_col='country', time_col='quarter',
                                  test=['hadri', 'breitung'], trend='c')

# Test with constant and trend
result_ct = panel_unit_root_test(df, 'real_exchange_rate',
                                   entity_col='country', time_col='quarter',
                                   test=['hadri', 'breitung'], trend='ct')

print("=" * 80)
print("Comparison: Constant vs. Constant + Trend")
print("=" * 80)
print("\nWith Constant Only:")
print(result_c.summary_table())
print("\n" + "=" * 80)
print("\nWith Constant + Trend:")
print(result_ct.summary_table())

## 6. Power Analysis: Unit Root vs. Stationary Data

Let's compare test results for clearly stationary vs. unit root data:

In [ ]:
# Generate clearly stationary data (ρ = 0.7)
df_stationary = generate_exchange_rate_data(mean_reversion=0.7)

# Generate data with unit root (ρ = 1.0)
df_unit_root = generate_exchange_rate_data(mean_reversion=1.0)

# Test both
print("=" * 80)
print("TEST 1: Clearly Stationary Data (ρ = 0.7)")
print("=" * 80)
result_stat = panel_unit_root_test(df_stationary, 'real_exchange_rate',
                                     entity_col='country', time_col='quarter',
                                     test='all', trend='c')
print(result_stat.summary_table())

print("\n" + "=" * 80)
print("TEST 2: Unit Root Data (ρ = 1.0)")
print("=" * 80)
result_ur = panel_unit_root_test(df_unit_root, 'real_exchange_rate',
                                   entity_col='country', time_col='quarter',
                                   test='all', trend='c')
print(result_ur.summary_table())

## 7. Interpretation Guidelines

### Understanding Different Null Hypotheses

**Unit Root Tests (Breitung, IPS, LLC):**
- H0: Series has unit root (non-stationary)
- Reject H0 → Evidence of stationarity

**Stationarity Test (Hadri):**
- H0: Series is stationary
- Reject H0 → Evidence of unit root

### Decision Rules

| Scenario | Unit Root Tests | Hadri Test | Conclusion |
|----------|----------------|------------|------------|
| 1 | Reject H0 | Don't reject H0 | **Stationary** |
| 2 | Don't reject H0 | Reject H0 | **Unit Root** |
| 3 | Reject H0 | Reject H0 | Mixed - proceed with caution |
| 4 | Don't reject H0 | Don't reject H0 | Inconclusive |

### Practical Recommendations

1. **If stationary**: Proceed with level regressions (fixed effects, random effects, etc.)
2. **If unit root**: Consider:
   - First differencing
   - Cointegration analysis
   - Error correction models
3. **If mixed**: 
   - Check robustness with different specifications
   - Consider economic theory
   - Try intermediate approaches (e.g., mean-differencing)

## 8. Real-World Application: PPP Testing

### Economic Context

Purchasing Power Parity (PPP) theory predicts:
- Real exchange rate = Nominal exchange rate × (Foreign price / Domestic price)
- Under PPP: Real exchange rate should be stationary

### Test Strategy

1. Compute real exchange rates
2. Test for unit roots
3. Interpret in light of PPP theory

In [ ]:
# Back to our original data (ρ = 0.95)
result_final = panel_unit_root_test(
    df,
    variable='real_exchange_rate',
    entity_col='country',
    time_col='quarter',
    test='all',
    trend='c'
)

print("=" * 80)
print("PPP Test Results")
print("=" * 80)
print(result_final.summary_table())
print("\n" + "=" * 80)
print("Economic Interpretation:")
print("=" * 80)
print("""
Our data shows evidence of mean reversion (stationarity), which is consistent
with PPP holding in the long run. However, the slow mean reversion (ρ = 0.95)
suggests that deviations from PPP are highly persistent.

This is consistent with empirical findings:
- PPP holds in the very long run (decades)
- Half-life of PPP deviations: ~3-5 years
- Short-run deviations can be large and persistent
""")

## 9. Summary and Best Practices

### Key Takeaways

1. **Use multiple tests**: Different tests have different power properties
2. **Check robustness**: Try different trend specifications (c vs. ct)
3. **Understand H0**: Hadri test has opposite null from traditional tests
4. **Consider economics**: Statistical evidence + economic theory
5. **Panel advantage**: Panel unit root tests have more power than individual time series tests

### When to Use Each Test

- **Breitung**: Recommended when heterogeneity across entities is a concern
- **Hadri**: Use as complementary test (reversed null hypothesis)
- **IPS**: Good general-purpose test with heterogeneous alternatives
- **LLC**: More powerful when homogeneity assumption is valid

### Next Steps

If you find evidence of unit roots:
1. Consider panel cointegration tests (Kao, Pedroni, Westerlund)
2. Estimate panel VECM (Vector Error Correction Model)
3. Use first differences if no cointegration

## References

1. Hadri, K. (2000). "Testing for Stationarity in Heterogeneous Panel Data." *Econometrics Journal*, 3(2), 148-161.

2. Breitung, J. (2000). "The Local Power of Some Unit Root Tests for Panel Data." In *Advances in Econometrics*, Vol. 15, 161-177.

3. Im, K. S., Pesaran, M. H., & Shin, Y. (2003). "Testing for Unit Roots in Heterogeneous Panels." *Journal of Econometrics*, 115(1), 53-74.

4. Levin, A., Lin, C. F., & Chu, C. S. J. (2002). "Unit Root Tests in Panel Data: Asymptotic and Finite-Sample Properties." *Journal of Econometrics*, 108(1), 1-24.